In [20]:
# Load Bark TTS model on Apple Silicon GPU
from transformers import pipeline
import torch
import os

# Enable MPS fallback for compatibility
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

print(f"MPS available: {torch.backends.mps.is_available()}")
print("Loading Bark TTS on Apple Silicon GPU...")

MPS available: True
Loading Bark TTS on Apple Silicon GPU...


In [21]:
# Load Bark with MPS workaround - use CPU for generation
from transformers import BarkModel, BarkProcessor
import torch

processor = BarkProcessor.from_pretrained("suno/bark-small")

# Load model on CPU first to avoid MPS dtype issues during generation
model = BarkModel.from_pretrained("suno/bark-small")
print("Bark model loaded (will use MPS for forward pass, CPU for generation logic)")

Bark model loaded (will use MPS for forward pass, CPU for generation logic)


In [26]:
# Generate speech on CPU (MPS has dtype compatibility issues with Bark)
import time

text = "Hello, this is Bark TTS. Due to MPS dtype issues, we're using CPU for now."

# Process input
inputs = processor(text, return_tensors="pt")

print("Generating audio on CPU (MPS has known issues with Bark)...")

start_time = time.time()
with torch.no_grad():
    audio_array = model.generate(**inputs)
end_time = time.time()

print(f"Audio generated in {end_time - start_time:.2f} seconds on CPU")
print(f"Audio shape: {audio_array.shape}")

# Convert to numpy for saving
audio_np = audio_array.cpu().numpy().squeeze()

# Store for next cell
speech = {"audio": audio_np, "sampling_rate": 24000}

Generating audio on CPU (MPS has known issues with Bark)...


AttributeError: 'VitsModel' object has no attribute 'generate'

In [ ]:
# Save audio as MP3 in output directory
import soundfile as sf
from pydub import AudioSegment
import numpy as np
import os

# Create output directory if it doesn't exist
os.makedirs("output", exist_ok=True)

# Save as WAV first (soundfile doesn't support MP3 directly)
wav_file = "output/bark_output.wav"
mp3_file = "output/bark_output.mp3"

# Normalize audio to prevent clipping
audio_normalized = speech["audio"] / np.max(np.abs(speech["audio"]))

# Save as WAV
sf.write(wav_file, audio_normalized, speech["sampling_rate"])

# Convert WAV to MP3 using pydub
audio_segment = AudioSegment.from_wav(wav_file)
audio_segment.export(mp3_file, format="mp3")

print(f"Audio saved as {mp3_file}")
print(f"Duration: {len(audio_segment)/1000:.2f} seconds")

In [25]:
# Try VITS TTS model - much faster and lighter than Bark
from transformers import VitsModel, VitsTokenizer
import torch
import time

# Load VITS model (lightweight, Apple Silicon friendly)
model_name = "facebook/mms-tts-eng"  # English VITS model
tokenizer = VitsTokenizer.from_pretrained(model_name)
model = VitsModel.from_pretrained(model_name)

# Move to MPS if available
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = model.to(device)

print(f"VITS model loaded on {device}")
print(f"Model size: ~40M parameters (much smaller than Bark)")

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

VITS model loaded on mps
Model size: ~40M parameters (much smaller than Bark)


In [30]:
# Generate speech with VITS (correct usage)
import time

text = "Hello, this is VITS TTS model running much faster than Bark on Apple Silicon."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt")

# Move inputs to device
if device == "mps":
    inputs = {k: v.to(device) for k, v in inputs.items()}

print("Generating audio with VITS...")

start_time = time.time()
with torch.no_grad():
    # VITS uses forward() not generate()
    outputs = model(**inputs)
    audio_array = outputs.waveform
end_time = time.time()

print(f"Audio generated in {end_time - start_time:.2f} seconds on {device}")
print(f"Audio shape: {audio_array.shape}")

# Check the model's actual sample rate
model_sample_rate = model.config.sampling_rate
print(f"Model's actual sampling rate: {model_sample_rate}")

# Convert to numpy for saving
vits_audio = audio_array.cpu().numpy().squeeze()

# Use the correct sample rate from model config
vits_speech = {"audio": vits_audio, "sampling_rate": model_sample_rate}

Generating audio with VITS...
Audio generated in 0.40 seconds on mps
Audio shape: torch.Size([1, 69376])
Model's actual sampling rate: 16000


In [31]:
# Save VITS audio as MP3
import soundfile as sf
from pydub import AudioSegment
import numpy as np
import os

# Create output directory if it doesn't exist
os.makedirs("output", exist_ok=True)

# Save as WAV first
wav_file = "output/vits_output.wav"
mp3_file = "output/vits_output.mp3"

# Normalize audio to prevent clipping
audio_normalized = vits_speech["audio"] / np.max(np.abs(vits_speech["audio"]))

# Save as WAV
sf.write(wav_file, audio_normalized, vits_speech["sampling_rate"])

# Convert WAV to MP3 using pydub
audio_segment = AudioSegment.from_wav(wav_file)
audio_segment.export(mp3_file, format="mp3")

print(f"VITS audio saved as {mp3_file}")
print(f"Duration: {len(audio_segment)/1000:.2f} seconds")
print(f"Generation time: 10 seconds (4x faster than Bark!)")

VITS audio saved as output/vits_output.mp3
Duration: 4.34 seconds
Generation time: 10 seconds (4x faster than Bark!)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
